This notebook has lots of very unpolished code that was used to visualize results. Proceed with caution.

In [ ]:
import os
import numpy as np

def get_max_displacement_in_folder(folder):
    files = os.listdir(folder)
    maxes = []
    bad_idx = []
    for i, file in enumerate(files):
        name = os.path.join(folder, file)
        data = np.load(name)
        maxval = np.max(np.abs(data["disp"][:,2]))
        maxes.append(maxval)
        print(i, end="\r")
        if maxval > 0.5:
            bad_idx.append(i)
    maxes = np.array(maxes)
    return maxes, bad_idx
    
maxes, bad = get_max_displacement_in_folder("../data-train")
maxes_te, bad_te = get_max_displacement_in_folder("../data-test")
maxes_val, bad_val = get_max_displacement_in_folder("../data-val")

In [2]:
import sys
sys.path.append("../")
import numpy as np
import importlib
from models_gpu import * # Modified
from coarsen import *
import torch
import torch.optim as opt
from visualize import *
from evaluate import *
from load_graphs import *
from visualize import *
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

In [3]:
device=torch.device("cuda:3" if torch.cuda.is_available() else "cpu")

In [ ]:
train_dataset = NetFabbDataset("../data-train", npz_mode=True, N=4, factor=8)
test_dataset = NetFabbDataset("../data-test", npz_mode=True, N=4, factor=8)
val_dataset = NetFabbDataset("../data-val", npz_mode=True, N=4, factor=8)


bad_names = [train_dataset.files[i] for i in bad]
bad_names_te = [test_dataset.files[i] for i in bad_te]
bad_names_val = [val_dataset.files[i] for i in bad_val]

for name in bad_names:
    train_dataset.files.remove(name)
for name in bad_names_te:
    test_dataset.files.remove(name)
for name in bad_names_val:
    val_dataset.files.remove(name)

train_loader = NetFabbDataLoader(train_dataset, shuffle=True, batch_size=8)
test_loader = NetFabbDataLoader(train_dataset, shuffle=True)
val_loader = NetFabbDataLoader(val_dataset, shuffle=True)

datasets = dict(tr=train_dataset, te=test_dataset) #, val=val_dataset)
model_names = ["model-gcn-unet","model-gnn-600k-f8-50-v2","model-600k-f8-75"] # "gnn-30-epoch","model-50-epoch","model-big-75","model-300k-50"

In [5]:
def export_scores_to_file(vals, path):
    with open(path, "w") as f:
        for key in vals:
            f.write(f"{key}\n")
            f.write(f"{len(vals[key])}\n")
            for val in vals[key]:
                f.write(f"{val}\n")

def read_scores_from_file(path):
    vals = dict()
    with open(path, "r") as f:
        lines = f.readlines()
    i = 0
    while i < len(lines):
        key = lines[i][:-1]
        i += 1
        count = int(lines[i])
        vals[key] = np.zeros(count)
        i += 1
        for j in range(count):
            vals[key][j] = float(lines[i])
            i += 1
    return vals


def sort_scores(vals):
    idx = dict()
    for key in vals:
        idx[key] = np.argsort(vals[key])
    return idx
    

def get_and_save_scores(model_filename):
    model = torch.load("../"+model_filename+".pth", map_location="cpu").to(device)
    scores = eval_model_multiple(model, datasets, device)
    export_scores_to_file(scores, "../results/results_"+model_filename+".txt")


In [7]:
# model_names = ["unet-2", "unet-4"]
# for name in model_names:
#     print(f"Starting for {name}")
#     get_and_save_scores(name)

#get_and_save_scores("model-gcn-unet")
get_and_save_scores("unet-1")
# get_and_save_scores("unet-4")

In [23]:
# model_names = ["unet-1","unet-2", "unet-3", "unet-4"]
model_names = ["model-gnn-600k-f8-50-v2", "model-gcn-unet", "unet-3"]

In [24]:
scores_list = dict()
scores_idx = dict()
models = dict()
for name in model_names:
    scores = read_scores_from_file("../results/results_"+name+".txt")
    scores_idx[name] = sort_scores(scores)
    scores_list[name] = scores
    models[name] = torch.load("../"+name+".pth", map_location="cpu").to(device)

get_best = lambda name, offset: scores_idx[name]["te"][-1-offset]
get_worst = lambda name, offset: scores_idx[name]["te"][0+offset]
get_median = lambda name, offset: scores_idx[name]["te"][len(datasets["te"])//2 + offset]

In [10]:


def get_vis(model_name, get = "best", offset = 0):
    model = models[model_name]
    if get == "best":
        get_fn = get_best
    elif get == "worst":
        get_fn = get_worst
    elif get == "median":
        get_fn = get_median

    i = get_fn(model_name, offset)

    data = test_dataset[i]


    verts = data.x.detach().numpy()
    gt = data.y.detach().numpy()
    pred = model(data, device).cpu().detach().numpy()

    thres=0.02

    print(f"{model_name}, {get}, {offset}")
    print(f"{datasets['te'].files[i][:-4]}")
    print(f"R2 = {scores_list[model_name]['te'][i]:.3f}")
    print(f"Accuracy = {100*accuracy_score(gt>thres, pred>thres):.1f}%")
    print(f"F1 Score = {f1_score(gt>thres, pred>thres):.3f}")




    plot_fields_custom(verts,[gt, pred,gt>thres,pred>thres], ["Ground Truth", "Prediction",f"G.T. > {thres}", f"Pred. > {thres}"], cmaps = ["jet", "jet", "bluered", "bluered"], limgroups = [0,0,1,1]) 
    return datasets['te'].files[i][:-4]

In [ ]:
name = "unet-3"
print(name)
get_vis(name, "median", -47) #-47
get_vis(name, "median", 96) #101

In [ ]:
for name in model_names:
    get_vis(name, "best", 0)
    get_vis(name, "median", 1)
    get_vis(name, "worst", 50)

In [ ]:
np.median(np.array(scores_list["model-600k-f8-75"]["te"]))

In [ ]:
#name = "model-600k-f8-75"
for name in model_names:
    print(f"{name}:")
    print(f"Train: {np.median(np.array(scores_list[name]['tr'])):.3f}")
    print(f"Test: {np.median(np.array(scores_list[name]['te'])):.3f}")

In [ ]:
model_str = dict()
model_str["model-gcn-unet"] = "TAG U-Net (GCNConv)"
model_str["unet-3"] = "TAG U-Net (EdgeConv)"
model_str["model-gnn-600k-f8-50-v2"] = "Plain GNN (EdgeConv)"

model_color=dict()
model_color["model-gcn-unet"] = "blue"
model_color["unet-3"] = "darkgreen"
model_color["model-gnn-600k-f8-50-v2"] = "darkred"


plt.figure(dpi=200)

for model in model_names:
    te = scores_list[model]["te"]
    log=True
    plt.hist(te[te>-1], bins=40, density=True, histtype="step", lw=3., edgecolor=model_color[model], label=model_str[model])
    #plt.hist(te[te>-1], bins=50, density=True, alpha=0.3, log=log, histtype="stepfilled", color=model_color[model])
plt.legend()
plt.xlabel("$R^2$")
plt.ylabel("Probability Density")
plt.title("3-D Displacement Prediction Task")
plt.show()

In [161]:
def plot_fields(xyz, fields, titles = None, cmap="coolwarm"):
    if titles is not None:
        assert len(fields) == len(titles)
    N = len(fields)

    vmin = min([min(f) for f in fields])
    vmax = max([max(f) for f in fields])

    fig = plt.figure(figsize=(3.5*N,4),dpi=300)
    for i, field in enumerate(fields):
        ax = fig.add_subplot(1, N, i+1, projection='3d')
        ax.scatter(xyz[:,0],xyz[:,1],xyz[:,2],c=field, cmap=cmap, vmin = vmin, vmax = vmax)
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_zticklabels([])
        equal_axes(ax)
        plt.title(titles[i])

    plt.show()

def plot_model_r2(model, data, lims=None, size=1, device="cpu"):
    pred = model(data, device).cpu().detach().numpy().flatten()
    gt = data.y.detach().numpy().flatten()
    plot_r2(gt, pred, lims=lims, size=size)

def plot_model_comparison(models, data, model_names, filename=None, dpi=300, size=13, device="cpu"):
    N = len(models)
    fig = plt.figure(figsize=(4.3*(N+1), 9.05), dpi=dpi)
    s = size# / (1 + 7*(3000<data.x.shape[0]))
    small_axes = []

    preds = dict()

    vmin = min(data.y.detach().numpy().flatten())
    vmax = max(data.y.detach().numpy().flatten())

    preds["gt"] = data.y.detach().numpy().flatten()
    model_names["gt"] = "Ground Truth"

    for key in models:
        model = models[key].to(device)
        pred = model(data, device=device).cpu().detach().numpy().flatten()
        preds[key] = pred
        vmin = min(vmin, min(pred))
        vmax = max(vmax, max(pred))

    

    xyz = data.x.detach().numpy()
    cmap = "jet"
    
    print(N+1)
    i = 0
    for key in preds:
        ax = fig.add_subplot(2, N+1, i+1, projection='3d')
        handle = ax.scatter(xyz[:,0],xyz[:,1],xyz[:,2],c=preds[key], cmap=cmap, vmin = vmin, vmax = vmax)
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_zticklabels([])
        equal_axes(ax)
        plt.title(model_names[key], fontsize=14)

        colorbar=True
        if colorbar:
            cbar_shrink = 0.93
            cbar_pad = -0.02
            ticks=[vmin, np.round((vmin + vmax)/2,3), vmax]
            bar = plt.colorbar(handle, shrink=cbar_shrink, location='bottom', pad=cbar_pad, ticks=ticks)
            bar.ax.set_xticklabels([np.round(vmin,3), np.round((vmin + vmax)/2,3), np.round(vmax,3)])

        if key != "gt":
            ax = plt.subplot(2,N+1,i+N+2)
            plot_r2(preds["gt"], preds[key])
            plt.ylabel("Prediction")
            plt.axis("scaled")
            small_axes.append(ax)

        i += 1

    

    plt.tight_layout()
    plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0.14, hspace=0)
    for ax in small_axes:
        pos1 = ax.get_position() # get the original position 
        pos2 = [pos1.x0 + 0.02, pos1.y0 + 0.1/N,  pos1.width * 0.8, pos1.height * 0.8] 
        ax.set_position(pos2) # set a new position

    if filename is not None:
        plt.savefig(filename, bbox_inches = "tight")
        plt.close()
    else:
        plt.show()

In [ ]:
for i in range(2):
    data = datasets['te'][i]
    print(datasets['te'].files[i][:-4])
    fig = plt.figure(dpi=200)
    xyz = data.x
    field = data.y
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(xyz[:,0],xyz[:,1],xyz[:,2],c=field, cmap="jet")
    equal_axes(ax)
    plt.axis("off")
    plt.show()


In [ ]:
idx = np.argsort(scores_list['unet-3']['te'])
imed = idx[len(idx)//2]
data = datasets['te'][imed]
print(datasets['te'].files[imed])

plot_model_comparison(models,data, model_str)

In [ ]:
plt.figure(dpi=100)
tr = scores_list["unet-3"]["tr"]
te = scores_list["unet-3"]["te"]
plt.hist(tr[tr>0], bins=100, density=True, label="Train")
plt.hist(te[te>0], bins=100, density=True, alpha=.5, label="Test")
plt.legend()
plt.xlabel("$R^2$")
plt.ylabel("Percent of shapes")
plt.show()

In [ ]:
plt.figure(dpi=100)
tr = scores_list["unet-3"]["tr"]
te = scores_list["unet-3"]["te"]
plt.hist(tr[tr>0], bins=100, density=True, label="Train")
plt.hist(te[te>0], bins=100, density=True, alpha=.5, label="Test")
plt.legend()
plt.xlabel("$R^2$")
plt.ylabel("Percent of shapes")
plt.show()

In [ ]:
plot_violins(scores_list["unet-3"])

In [ ]:
plot_boxes(scores_list["unet-3"])

In [44]:
from torch_geometric.data import Data
def get_rot_fn(w):
    Rs = torch.tensor([
         [[1,0,0],[0,1,0],[0,0,1]],
         [[1,0,0],[0,0,1],[0,-1,0]],
         [[0,0,1],[0,1,0],[-1,0,0]],
         [[1,0,0],[0,-1,0],[0,0,1]],
         [[1,0,0],[0,0,-1],[0,1,0]],
         [[0,0,-1],[0,1,0],[1,0,0]]], dtype=torch.float)
    R = Rs[w]
    def rotate(X):
        X = X @ R
        # Minimum z-value should be 0
        # x and y bounds should be centered at 0
        X[:,2] = X[:,2] - torch.min(X[:,2])
        medx = (torch.max(X[:,0])-torch.min(X[:,0]))/2.
        medy = (torch.max(X[:,1])-torch.min(X[:,1]))/2.
        X[:,0] = X[:,0] - medx
        X[:,1] = X[:,1] - medy
        return X
    return rotate


def rotate_shape(data, w):
    rotate = get_rot_fn(w)
    new = Data(x=rotate(data.x), y=data.y, edge_index=data.edge_index)
    return Coarsen(new, factor=data.factor, N=data.N)
    

In [113]:
import matplotlib.gridspec as gridspec

def plot_all_orientations(data, model, cmap="jet", title=""):
    fig = plt.figure(figsize=(12,6),dpi=200)
    mins = []
    maxs = []
    peaks = []
    for i in range(6):
        d = rotate_shape(data,i)
        mins.append(np.min(model(d, device).cpu().detach().numpy()))
        maxs.append(np.max(model(d, device).cpu().detach().numpy()))
        peaks.append(max([maxs[i], abs(mins[i])]))

    min_index = np.argmin(np.array(peaks))
    max_index = np.argmax(np.array(peaks))

    for i in range(6):
        ax = fig.add_subplot(2,3,i+1, projection='3d')
        d = rotate_shape(data,i)
        xyz = d.x.cpu().detach().numpy()
        scatter = ax.scatter(xyz[:,0],xyz[:,1],xyz[:,2],c=model(d, device).cpu().detach().numpy(), 
                             cmap=cmap, vmin=min(mins), vmax=max(maxs))
        #cbar = fig.colorbar(scatter, format="% .2f", shrink=0.8)
        if i == min_index:
            plt.title(f"BEST: Peak = {peaks[i]:.2f}")
        elif i == max_index:
            plt.title(f"WORST: Peak = {peaks[i]:.2f}")
        else:
            plt.title(f"Peak = {peaks[i]:.2f}")

        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_zticklabels([])
    plt.show()

In [ ]:
i = 100

data = datasets["te"][i]

plot_all_orientations(data,models[model_names[-1]])

In [150]:
def get_acc_scores(model, dataset, threshold = 0.02, device="cpu"):
    py_gy = 0
    py_gn = 0
    pn_gy = 0
    pn_gn = 0
    accs = []
    f1s = []

    for data in dataset:
        gt = data.y.cpu().detach().numpy().flatten()
        pred = model(data, device).cpu().detach().numpy().flatten()

        gt_bin = gt > threshold
        pred_bin = pred > threshold

        accs.append(accuracy_score(gt_bin, pred_bin))
        f1s.append(f1_score(gt_bin, pred_bin,))
        py_gy += np.sum(np.logical_and(gt_bin, pred_bin))
        py_gn += np.sum(np.logical_and(gt_bin, np.logical_not(pred_bin)))
        pn_gy += np.sum(np.logical_and( np.logical_not(gt_bin), pred_bin))
        pn_gn += np.sum(np.logical_and(np.logical_not(gt_bin),  np.logical_not(pred_bin)))

    results = dict(py_gy=py_gy, py_gn=py_gn, pn_gn=pn_gn, pn_gy=pn_gy, accs=np.array(accs), f1s=np.array(f1s))
    return results


In [ ]:
results = results_te
print("median",np.median(results["accs"]))
print("mean",np.mean(results["accs"]))
print("stdev",np.std(results["accs"]))

print("median",np.median(results["f1s"]))
print("mean",np.mean(results["f1s"]))
print("stdev",np.std(results["f1s"]))

print("                 GT <= 0.02,     GT > 0.02")
print(f"Pred <= 0.02:     {results['pn_gn']},   {results['pn_gy']}")
print(f"Pred > 0.02:     {results['py_gn']},   {results['py_gy']}")

In [ ]:
results_te = get_acc_scores(models["unet-3"], datasets["te"])
print("1 done.")
results_tr = get_acc_scores(models["unet-3"], datasets["tr"])

In [178]:
from PIL import Image

def auto_crop(image_file):
    
    image = Image.open(image_file)
    A = np.array(image)

    rows = A.shape[0]
    cols = A.shape[1]

    for i in range(cols):
        if len(np.unique(A[:,i])) > 1:
            A = A[:,i:]
            cols = A.shape[1]
            break

    for i in range(cols-1,0,-1):
        if len(np.unique(A[:,i])) > 1:
            A = A[:,:i]
            rows = A.shape[0]
            break

    for i in range(rows):
        if len(np.unique(A[i,:])) > 1:
            A = A[i:,:]
            rows = A.shape[0]
            break

    for i in range(rows-1,0,-1):
        if len(np.unique(A[i,:])) > 1:
            A = A[:i,:]
            break

    image = Image.fromarray(A)

    # Save the image
    image.save(image_file) 


def plot_index(i):
    data = datasets['te'][i]
    print(datasets['te'].files[i][:-4])
    fig = plt.figure(dpi=300)
    xyz = data.x
    field = data.y
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(xyz[:,0],xyz[:,1],xyz[:,2],c=field, cmap="jet")
    equal_axes(ax)
    plt.axis("off")
    plt.savefig("temp.png")
    plt.close()
    auto_crop("temp.png")
    img = Image.open("temp.png")
    #plt.figure(dpi=300)
    plt.imshow(np.array(img))
    plt.axis("off")
    plt.show()

    return datasets['te'].files[i][:-4]


In [ ]:
names = []
for i in range(20):
    names.append(plot_index(i))

In [ ]:
idx = np.argsort(scores_list['unet-3']['te'])
imed = idx[3*len(idx)//4-1]
i = imed
data = datasets['te'][i]
print(datasets['te'].files[i])

plot_model_comparison(models,data, model_str)

In [ ]:
names